In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sb
import os
import glob
import json 
import requests
from bs4 import BeautifulSoup
%matplotlib inline

In [2]:
# os.listdir('./Prediction_train&test_Data')
def load_data(url):
    
    folderName = "./Prediction_train&test_Data/"
    fileName = "Store_league_data.json"
    path = os.path.join(folderName, fileName)

#     if os.path.exists(path):
#         files = glob.glob("./Prediction_train&test_Data/*")
#         for f in files:
#             os.remove(f)

    response = requests.get(url)
    # print(path)

    with open(path, 'wb') as outfile:
        outfile.write(response.content)
     

        
url = "https://vsmobile.bet9ja.com/shopadmin/standings_view/league_data.php?pid=14001"
load_data(url)

In [3]:
folderName = "./Prediction_train&test_Data/"
fileName = "Store_league_data.json"


path = os.path.join(folderName, fileName)


with open(path, "r") as outfile:
    json_obj = json.load(outfile)

In [4]:
# print(json_obj)

In [5]:
# json_obj["matches"]["1"].keys() # 1539761

In [6]:
# json_obj["matches"]["1"]["1540521"]["wonmarkets"].split(",")

In [7]:
# home_away_game = json_obj["matches"]["1"]["1537861"]["wonmarkets"].split(",")[2].split("_")
# games_scores = json_obj["matches"]["1"]["1537861"]["wonmarkets"].split(",")[4].split("_")
# total_goales = json_obj["matches"]["1"]["1537861"]["wonmarkets"].split(",")[5].split("_")

# print( f"games: {home_away_game}, games score: {games_scores}, scores_total: {total_goales}" )

In [8]:
# len(json_obj["matches"].keys()), len(json_obj["matches"]["1"].keys())
json_obj["matches"]["1"].keys()

dict_keys(['1555721', '1555722', '1555723', '1555724', '1555725', '1555726', '1555727', '1555728', '1555729', '1555730'])

In [9]:
teams = {"id": [], "name": [], "club": []}

In [10]:
for x, y in json_obj["teams"].items():
    # main_info.id.append(x)
    teams["id"].append(x)
    teams["name"].append(y["fifa"])
    teams["club"].append(y["team"])
   
    
    # print(x)
    # print(y)

In [11]:
# teams # if you have ound its index you havefound the club

In [12]:
# len(teams['id']) # total number of teams 

In [13]:
# len(json_obj["weeks"])

In [14]:
# print(json_obj["weeks"]["1"]["event_data"]), print(len(json_obj["weeks"]["1"]["event_data"]))

In [15]:
# print(json_obj["weeks"]["1"]["teamstats"].split("-"))

In [16]:
def indetify_club(teams_id):
    idx = teams["id"].index(teams_id)
    return teams["name"][idx]

def get_club_id(team_name):
    club_id = teams["name"].index(team_name)
    return teams["id"][club_id]

In [17]:
# indetify_club("50")

In [18]:
# json_obj["weeks"]["1"]

In [19]:
def query_score(teamA, teamB):
    if teamA > teamB:
        return "win-loss"
    elif teamA < teamB:
        return "loss-win"
    else:
        return "draw-draw"

In [20]:
# json_obj["matches"]['1']['1537862']# .keys()

In [21]:
def get_new_data(data, score, week, history):
    start = 0
    stop = 1
    add_num = 2
    data = data.split("-")
    score = score.split("-")
    
    # store clubs id and name.
    result = {
        "weeks": [],
        "id":[],
        "club": [],
        "score_string": [],
        "score_num": [],
        "game_type": [],
        "team_stats": [],
        "team_history": []
    }
    
    # GET ALL WEEK GAME ID
    games_id = list(json_obj["matches"][week].keys())
    
    for i in range(10):
        
        # GET GAME RESULTES
        game_resulte = json_obj["matches"][week][games_id[i]]["wonmarkets"]
        
        # slice the event_data from data pass thoure the function parameter
        teamA = data[start]
        teamB = data[stop]

        # identfy the club from thire respected ID
        clubA = indetify_club(teamA)
        clubB = indetify_club(teamB)
		
        # slice the teamstats from score pass thoure the function parameter
        teamA_score = game_resulte.split(",")[4].split("_")[1] # score[start]
        teamB_score = game_resulte.split(",")[4].split("_")[2] # score[stop] 
        
        # team stats
        teamstatsA = score[start]
        teamstatsB = score[stop]
        
        # team history
        team_histA = sum([ int(j) for j in history.split("-")[start].replace("x", "")])
        team_histB = sum([ int(j) for j in history.split("-")[stop].replace("x", "")])
        
        result["weeks"].append( int(week) )
        result["weeks"].append( int(week) )
        
		# appending the data twice to add information
        
        ####### HOME APPENDING #######
        result["id"].append( int(teamA) )
        
        result["club"].append(f"{clubA}")
        # query scores
        win_loss_drawA = query_score( int(teamA_score), int(teamB_score) ).split("-")[0]
        
        result["score_string"].append( win_loss_drawA )
        # query wins scores
        result["score_num"].append( f" {int(teamA_score)} " )
        
        result["game_type"].append( "home" )
        
        result["team_stats"].append( int(teamstatsA) )
        
        result["team_history"].append(team_histA)
        
        
        ####### AWAY APPENDING #######
        result["id"].append( int(teamB) )
        
        result["club"].append(f"{clubB}")
        
        win_loss_drawB = query_score( int(teamA_score), int(teamB_score) ).split("-")[1]
        
        result["score_string"].append( win_loss_drawB )
        
        result["score_num"].append( f" {int(teamB_score)} " )
        
        result["game_type"].append( "away" )
        
        result["team_stats"].append( int(teamstatsB) )
        
        result["team_history"].append(team_histB)
        
        
        # accumilate the START and STOP for the next sclice
        start += add_num
        stop += add_num

        
        
    return result
 

In [22]:
# get_new_data( json_obj["weeks"]["1"]["event_data"], json_obj["weeks"]["1"]["teamstats"],  json_obj["weeks"]["1"]["matchday"])

# games_id = list(json_obj["matches"][week].keys())
# game_resulte = json_obj["matches"]["1"][games_id[1]]
# game_resulte

In [23]:
json_obj["weeks"]["4"]["event_data"]

'63-50-47-52-1053-827-525-46-58-49-48-55-512-60-53-972-45-51-71-67'

In [24]:
main_second_info = []
for week, events in json_obj["weeks"].items():
    main_second_info.append(get_new_data( events["event_data"], events["teamstats"], events["matchday"], events["teamhistory"]) )
    
    # print(events["matchday"])

In [25]:
# json_obj["weeks"]

In [26]:
# main_second_info[0]

# create database

In [27]:
df_list = []
for data in main_second_info:
    new_data = pd.DataFrame(data)
    df_list.append(new_data)

In [28]:
df = pd.concat(df_list)

In [29]:
df.sample(20)

,weeks,id,club,score_string,score_num,game_type,team_stats,team_history
2,9,46,MNC,win,3,home,16,9
17,4,51,TOT,loss,1,away,5,5
15,8,512,FUL,draw,1,away,3,2
12,3,67,BOU,win,2,home,3,3
4,9,60,ASV,loss,1,home,10,4
11,2,51,TOT,draw,1,away,4,4
19,16,512,FUL,loss,1,away,11,4
16,30,45,LIV,win,3,home,69,12
9,1,58,WHU,loss,1,away,0,0
18,17,52,SOU,loss,0,home,25,7


### Wrangling Data

In [30]:
df["binary_game_type"] = df["game_type"].apply(lambda x: 0 if x == "home" else 1)

In [31]:
df.head()

,weeks,id,club,score_string,score_num,game_type,team_stats,team_history,binary_game_type
0,1,49,EVE,win,2,home,3,3,0
1,1,71,BRI,loss,0,away,0,0,1
2,1,60,ASV,loss,1,home,0,0,0
3,1,52,SOU,win,2,away,3,3,1
4,1,972,BRN,loss,0,home,0,0,0


In [32]:
# df["weeks"].apply(lambda x: int(x))

### ANALYING DATA

In [33]:
# color_paletter = sb.color_palette()[0]
# sb.barplot(x = df["score_string"], y = df["weeks"].apply(lambda x: int(x)) , color=color_paletter)

In [34]:
# df.groupby("club").mean()
# df.head()

In [35]:
# sb.barplot(x = df["club"], y = df["score_string"] , color=color_paletter)
all_tems = df[["club", "score_string"]].value_counts()
# df.query("club == 'MNU'")
club = all_tems.index
club_values = all_tems.values
# plt.figure(figsize=[10, 6])
main_club = [x[0] for x in club]
main_scores = [x[1] for x in club]
print(f"main_club: {main_club[0]}, --- main_scores: {main_scores[0]}, ---, club_values: {club_values[0]}")

main_club: LIV, --- main_scores: win, ---, club_values: 23


In [36]:
query_df_club = pd.DataFrame({
    "clubs": main_club,
    "scores": main_scores,
    "occ": club_values
})

In [37]:
# query_df_club

In [38]:
# plt.figure(figsize=[20, 10])
# sb.barplot(data=query_df_club, x="clubs", y="occ", hue="scores")

In [39]:
query_df_club_wins = query_df_club.query("scores == 'win'")
query_df_club_loss = query_df_club.query("scores == 'loss'")
query_df_club_draw = query_df_club.query("scores == 'draw'")
# query_df_club_draw.head()

In [40]:
# plt.figure(figsize=[20, 10])
# plt.title(" WINS ")
# sb.barplot(data=query_df_club_wins, x="clubs", y="occ", color=color_paletter)

In [41]:
# plt.figure(figsize=[20, 10])
# plt.title(" LOSS ")
# sb.barplot(data=query_df_club_loss, x="clubs", y="occ", color=color_paletter)

In [42]:
# df.head()

In [43]:
# plt.figure(figsize=[20, 10])
# plt.title(" DRAWS ")
# sb.barplot(data=query_df_club_draw, x="clubs", y="occ", color=color_paletter)

In [44]:
# sort_count = df.score_string.value_counts()

# plt.figure(figsize=[20, 10])

# plt.pie([np.float64(x) for x in sort_count.values], labels=sort_count.index, startangle=90, counterclock=False);


### Select training data

In [45]:
df.head()

,weeks,id,club,score_string,score_num,game_type,team_stats,team_history,binary_game_type
0,1,49,EVE,win,2,home,3,3,0
1,1,71,BRI,loss,0,away,0,0,1
2,1,60,ASV,loss,1,home,0,0,0
3,1,52,SOU,win,2,away,3,3,1
4,1,972,BRN,loss,0,home,0,0,0


In [46]:
train_df = df[["weeks", "id", "score_string", "binary_game_type", "team_stats", "team_history"]]
# train_df = df[["club", "final"]]

#### STANDARDIZATION OF TRAIN DATA FRAME

In [47]:
# train_df[["weeks_normalization"]]
standardized_col = ["standardized_weeks", "standardized_id", "standardized_team_stats", "standardized_team_history"]
col = ["weeks", "id", "team_stats", "team_history"]
for x in range(len(standardized_col)):
    ################ WEEK STANDADIZATION BY VARIANCE ################
    # train_df[standardized_col[x]] = (train_df[col[x]] - train_df[col[x]].mean())/train_df[col[x]].std()**2
    ################ WEEK STANDADIZATION BY STANDARD DIVIATION ################
    train_df[standardized_col[x]] = (train_df[col[x]] - train_df[col[x]].mean())/train_df[col[x]].std()
    
# (train_df["weeks"] - train_df["weeks"].mean())/train_df["weeks"].std()

<ipython-input-47-f6b993423e6a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[standardized_col[x]] = (train_df[col[x]] - train_df[col[x]].mean())/train_df[col[x]].std()


In [48]:
train_df.sample(5)

,weeks,id,score_string,binary_game_type,team_stats,team_history,standardized_weeks,standardized_id,standardized_team_stats,standardized_team_history
19,23,58,loss,1,34,6,0.781992,-0.532148,0.798060,-0.120800
12,10,58,loss,0,16,7,-0.670279,-0.532148,-0.418654,0.170625
6,8,827,loss,0,4,3,-0.893706,1.777232,-1.229797,-0.995076
11,17,45,draw,1,44,11,0.111713,-0.571189,1.474012,1.336326
9,24,827,loss,1,26,9,0.893706,1.777232,0.257298,0.753475


In [49]:
train_df = train_df.reset_index()
train_df = train_df.drop('index', axis = 1) 
# train_df.head()

In [50]:
train_df.head()

,weeks,id,score_string,binary_game_type,team_stats,team_history,standardized_weeks,standardized_id,standardized_team_stats,standardized_team_history
0,1,49,win,0,3,3,-1.675698,-0.559176,-1.297392,-0.995076
1,1,71,loss,1,0,0,-1.675698,-0.493108,-1.500178,-1.869352
2,1,60,loss,0,0,0,-1.675698,-0.526142,-1.500178,-1.869352
3,1,52,win,1,3,3,-1.675698,-0.550167,-1.297392,-0.995076
4,1,972,loss,0,0,0,-1.675698,2.212680,-1.500178,-1.869352


#### Train the model

In [51]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

#### SPLIT DATA FOR EVEN FEATURES

In [52]:
# train_df[["home", "away"]][:20]["home"].value_counts()
# prec_20 = int((20/100)*len(train_df))
# prec_20

In [53]:
# train_df[["home", "away"]][:20]["away"].value_counts()

In [54]:
# train_data = train_df[prec_20:]
# test_data = train_df[:prec_20]

In [55]:
# train_data.head()

### Ordinary Split

In [56]:
# x_train_data = train_data[["id"]]
# x_train_data_dummy = pd.get_dummies(x_train_data)
# y_train_data = train_data["score_string"]

In [57]:
# x_test_data = test_data[["id"]]
# x_test_data_dummy = pd.get_dummies(x_test_data)
# y_test_data = test_data["score_string"]

### cross validation spilt

In [58]:
# x_train, x_test, y_train, y_test = train_test_split(train_df[["id"]], train_df["score_string"], test_size=0.3)
# x_train, x_test, y_train, y_test = train_test_split(train_df["club"], train_df["final"], test_size=0.3)

### Stratified Shuffle Split

In [59]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2)
for train_index, test_index in split.split(train_df[["standardized_weeks", "standardized_id", "standardized_team_stats", "binary_game_type", "standardized_team_history"]], train_df["score_string"]):
    strat_train_set = train_df.loc[train_index]
    strat_test_set = train_df.loc[test_index]    

In [60]:
# y_train.head()

In [61]:
# x_train.head()

In [62]:
# trat_train_set.head()

In [63]:
# strat_test_set.head()

### StratifiedShuffleSplit train

In [64]:
y_strat_train_set = strat_train_set["score_string"]
x_strat_train_set = strat_train_set[["standardized_weeks", "standardized_id", "standardized_team_stats", "standardized_team_history", "binary_game_type"]]
# x_strat_train_set = strat_train_set["club"]

In [65]:
# len(y_strat_train_set), len(x_strat_train_set)

### StratifiedShuffleSplit test

In [66]:
y_strat_test_set = strat_test_set["score_string"]
x_strat_test_set = strat_test_set[["standardized_weeks", "standardized_id", "standardized_team_stats", "standardized_team_history", "binary_game_type"]]
# x_strat_test_set = strat_test_set["club"]

In [67]:
# len(y_strat_train_set), len(x_strat_train_set)

In [68]:
# x_strat_train_set, x_strat_test_set, y_strat_train_set, y_strat_test_set

### Selecting and Training model

##### Decision tree classifer

In [69]:
model = DecisionTreeClassifier()
model.fit(x_strat_train_set, y_strat_train_set)
# x_strat_train_set, x_strat_test_set, y_strat_train_set, y_strat_test_set

DecisionTreeClassifier()

In [70]:
predictions = model.predict(x_strat_test_set)


In [71]:
# len(predictions) #, len(y_strat_test_set), len(x_strat_test_dummy)
# y_strat_test_set

In [72]:
accuracy = accuracy_score(y_strat_test_set, predictions)

In [73]:
print(f"accuracy: {accuracy}")

accuracy: 0.5080645161290323


##### SGDClassifier

In [74]:
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(x_strat_train_set, y_strat_train_set)
# x_strat_train_set, x_strat_test_set, y_strat_train_set, y_strat_test_set

SGDClassifier(random_state=42)

In [75]:
sgd_predict = sgd_clf.predict(x_strat_test_set)
sgd_accuracy = accuracy_score(y_strat_test_set, sgd_predict)

In [76]:
print(f"accuracy: {sgd_accuracy}")

accuracy: 0.47580645161290325


##### RandomForestClassifier & cross_validation

In [77]:
forest_clf = RandomForestClassifier(random_state=42)

In [78]:
from sklearn.model_selection import cross_val_score
main_data = cross_val_score(sgd_clf, x_strat_train_set, y_strat_train_set, cv=7, scoring="accuracy")
print(main_data)
# x_strat_train_set, x_strat_test_set, y_strat_train_set, y_strat_test_set

[0.54929577 0.54929577 0.36619718 0.54929577 0.52112676 0.45070423
 0.48571429]


In [79]:
main_data_forst_clf = cross_val_score(forest_clf, x_strat_train_set, y_strat_train_set, cv=7, scoring="accuracy")
# x_train_data_dummy, y_train_data, x_test_data_dummy, y_test_data
main_data_forst_clf

array([0.45070423, 0.47887324, 0.43661972, 0.49295775, 0.49295775,
       0.52112676, 0.5       ])

In [80]:
main_data.mean(), main_data_forst_clf.mean()

(0.49594711123886176, 0.4818913480885311)

#### Gride Search Cross Validation

In [81]:
from sklearn.model_selection import GridSearchCV

#### Random Forest Gride Search Cross Validation 

In [82]:
param_grid = [
    {"n_estimators": [3, 10, 30], 
     # "max_features": [0, 0, 0]
    },
    {"bootstrap": [False], "n_estimators": [3, 10], 
     # "max_features": [2, 3, 4]
    }
]
grid_search = GridSearchCV(forest_clf, param_grid, cv=7, scoring="accuracy")

In [83]:
grid_search.fit(x_strat_train_set, y_strat_train_set)
# x_strat_train_set, x_strat_test_set, y_strat_train_set, y_strat_test_set

GridSearchCV(cv=7, estimator=RandomForestClassifier(random_state=42),
             param_grid=[{'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'n_estimators': [3, 10]}],
             scoring='accuracy')

In [84]:
grid_search.best_params_

{'n_estimators': 10}

In [85]:
final_model = grid_search.best_estimator_
final_model

RandomForestClassifier(n_estimators=10, random_state=42)

In [86]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(mean_score), params)

0.6659500795701191 {'n_estimators': 3}
0.6882373100890148 {'n_estimators': 10}
0.6824073512528849 {'n_estimators': 30}
0.676335934327082 {'bootstrap': False, 'n_estimators': 3}
0.6780125836029897 {'bootstrap': False, 'n_estimators': 10}


#### Support Vectore Maching Gride Search Cross Validation 

In [87]:
svm_clf = SVC()
svm_parames = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 
svm_gridCV = GridSearchCV(SVC(), svm_parames, refit = True, cv=7, scoring="accuracy")

svm_gridCV.fit(x_strat_train_set, y_strat_train_set)

GridSearchCV(cv=7, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             scoring='accuracy')

In [88]:
svm_gridCV.best_params_

{'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}

In [89]:
final_model_svc = svm_gridCV.best_estimator_
final_model_svc

SVC(C=1000, gamma=0.01)

In [90]:
svm_cvres = svm_gridCV.cv_results_
for mean_score, params in zip(svm_cvres["mean_test_score"], svm_cvres["params"]):
    print(np.sqrt(mean_score), params)

0.7211740292427019 {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
0.7308934473562682 {'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'}
0.7197976465031466 {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}
0.6881537760056766 {'C': 0.1, 'gamma': 0.001, 'kernel': 'rbf'}
0.6866902846234707 {'C': 0.1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.7141247303498663 {'C': 1, 'gamma': 1, 'kernel': 'rbf'}
0.7419007928509599 {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
0.7295551019200456 {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
0.7239785005200718 {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.6866902846234707 {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.6584188469220629 {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
0.7378604589710076 {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
0.743274917348496 {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
0.7253865766209231 {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.7253865766209231 {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.6058306330686469 {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
0.7155722819643889 {'C': 100, 

### FINAL PREDICTION!!!🎉👏🏽🎊

In [91]:
# x_train_data_dummy, y_train_data, x_test_data_dummy, y_test_data
# x_strat_train_set, x_strat_test_set, y_strat_train_set, y_strat_test_set

final_predctions = final_model.predict(x_strat_test_set)
final_accuracy_score = accuracy_score(y_strat_test_set, final_predctions)
print(f"accuracy: {final_accuracy_score}")

accuracy: 0.5161290322580645


In [92]:
svc_final_predctions = final_model_svc.predict(x_strat_test_set)
svc_final_accuracy_score = accuracy_score(y_strat_test_set, svc_final_predctions)
print(f"accuracy: {svc_final_accuracy_score}")

accuracy: 0.5483870967741935


#### Main predicions!! 

In [93]:
id_cubs = np.array([get_club_id(x) for x in df.club.value_counts().index]).reshape(-1, 1)
cubs = df.club.value_counts().index

##### SELECTING MAIN TEAMS FOR PREDICTIONS

In [94]:
df.club.value_counts().index

Index(['CHE', 'MNU', 'ARS', 'TOT', 'EVE', 'WHU', 'SOU', 'NWC', 'LEE', 'CRY',
       'BRN', 'BRI', 'ASV', 'FUL', 'LEI', 'MNC', 'FOR', 'LIV', 'WOL', 'BOU'],
      dtype='object')

In [95]:
main_clubs = ["BRI", "LEE", "BRI", "LEE"]
teams_id = [get_club_id(x) for x in main_clubs]
game_type = ["home", "home", "away", "away"]
weaks = np.array([(len(df)/20)+1 for _ in range(4)])
################ WEEK STANDADIZATION BY VARIANCE ################
# standardized_weeks = (weaks-train_df["weeks"].mean())/train_df["weeks"].std()**2
################ WEEK STANDADIZATION BY STANDARD DIVIATION ################
standardized_weeks = (weaks-train_df["weeks"].mean())/train_df["weeks"].std()

In [96]:
x_strat_train_set.head(3)

,standardized_weeks,standardized_id,standardized_team_stats,standardized_team_history,binary_game_type
450,0.781992,2.212680,0.933250,0.753475,0
79,-1.340558,-0.505121,-1.297392,-0.995076,1
225,-0.446853,-0.550167,-0.283464,0.170625,1


In [97]:
train_df.head(3)

,weeks,id,score_string,binary_game_type,team_stats,team_history,standardized_weeks,standardized_id,standardized_team_stats,standardized_team_history
0,1,49,win,0,3,3,-1.675698,-0.559176,-1.297392,-0.995076
1,1,71,loss,1,0,0,-1.675698,-0.493108,-1.500178,-1.869352
2,1,60,loss,0,0,0,-1.675698,-0.526142,-1.500178,-1.869352


#### Standardized week, standardized clube ids, and standardized team stats

In [98]:
# train_df.query(f"id == {get_club_id('BRI')}").query(f"weeks == { int(len(df)/20) }").standardized_team_stats.values[0]

In [99]:
# main_clubs = ["BRI", "LEE", "WOL", "BRN"]
predict_teams_stats = [] 
predict_standardized_team_stats = []
predict_standardized_club_id = []
predict_team_history = []
predict_standardized_team_history = []

for i in range(len(main_clubs)):
    predict_teams_stats.append(train_df.query(f"id == {get_club_id(main_clubs[i])}").query(f"weeks == { int(len(df)/20) }").team_stats.values[0])
    predict_standardized_team_stats.append(train_df.query(f"id == {get_club_id(main_clubs[i])}").query(f"weeks == { int(len(df)/20) }").standardized_team_stats.values[0])
    predict_standardized_club_id.append(train_df.query(f"id == {get_club_id(main_clubs[i])}").query(f"weeks == { int(len(df)/20) }").standardized_id.values[0])
    predict_team_history.append(train_df.query(f"id == {get_club_id(main_clubs[i])}").query(f"weeks == { int(len(df)/20) }").team_history.values[0])
    predict_standardized_team_history.append(train_df.query(f"id == {get_club_id(main_clubs[i])}").query(f"weeks == { int(len(df)/20) }").standardized_team_history.values[0])

In [100]:
# current_teams_stats

##### Combine Data For DataFrame

In [101]:
main_prediction_data = pd.DataFrame({
    "weeks": weaks,
    "standardized_weeks": standardized_weeks,
    "club_id": teams_id,
    "standardized_club_id":  predict_standardized_club_id,
    "team_stats": predict_teams_stats,
    "standardized_team_stats": predict_standardized_team_stats,
    "binary_game_type": [ 0 if x == "home" else 1 for x in game_type ],
    "team_history": predict_team_history,
    "standardized_team_history": predict_standardized_team_history,
})


prediction_data = main_prediction_data[["standardized_weeks", "standardized_club_id", "standardized_team_stats", "standardized_team_history", "binary_game_type"]]
prediction_data

,standardized_weeks,standardized_club_id,standardized_team_stats,standardized_team_history,binary_game_type
0,1.787411,-0.493108,0.189703,-0.412226,0
1,1.787411,1.777232,0.865655,-0.120800,0
2,1.787411,-0.493108,0.189703,-0.412226,1
3,1.787411,1.777232,0.865655,-0.120800,1


In [102]:
rfc_predictions = final_model.predict(prediction_data)
svm_predictions = final_model_svc.predict(prediction_data)

In [103]:
# [get_club_id(x) for x in df.club.value_counts().index]

In [104]:
# plt.figure(figsize=[20, 10])
# sb.barplot(data=query_df_club, x="clubs", y="occ", hue="scores")
# plt.xlabel(fontsize=25)
# plt.xlabel('xlabel', fontsize=18);

In [105]:
prediction_df = pd.DataFrame({
    "week": weaks,
    "id_clubs": teams_id,
    "clubs": main_clubs,
    "rf_predictions": rfc_predictions,
    "svm_predictios": svm_predictions,
    "game_type": game_type
})
prediction_df["rf_accuracy"] = final_accuracy_score
prediction_df["svm_accuracy"] = svc_final_accuracy_score
# prediction_df["mean_accuracy"] = (final_accuracy_score+svc_final_accuracy_score)/2
prediction_df
# .sort_values(by=["clubs"])

,week,id_clubs,clubs,rf_predictions,svm_predictios,game_type,rf_accuracy,svm_accuracy
0,32.0,71,BRI,draw,draw,home,0.516129,0.548387
1,32.0,827,LEE,draw,draw,home,0.516129,0.548387
2,32.0,71,BRI,draw,loss,away,0.516129,0.548387
3,32.0,827,LEE,draw,draw,away,0.516129,0.548387


In [106]:
print(f"week of game {len(df)/20}")

week of game 31.0


In [107]:
# indetify_club
# new_df = pd.DataFrame({"home": [indetify_club("MNC"), indetify_club("LEI"), indetify_club("LIV"), indetify_club("NWC")], 
  #                     "away": [indetify_club("BRI"), indetify_club("CHE"), indetify_club("TOT"), indetify_club("FOR")]})

### PRE PER PREDICTIONS.

In [108]:
# d4 = BeautifulSoup
url = "https://bam.nr-data.net/jserrors/1/c95cd51526?a=214311961&v=1223.PROD&to=ZlBXZxcAVkEHV0NbDV8aYEEMTlpXEg1dU09cWldaCQQXXglTXlxNWFtRVh1PSFoW&rst=476457&ck=0&s=0&ref=https://vsmobile.bet9ja.com/bet9ja-mobile/login/"
html = requests.get(url)
print(html.text)

s = BeautifulSoup(html.content, "html.parser")
# requests

GIF89a     ,       


In [109]:
# url = https://vsmobile.bet9ja.com" + uri + (containsChar('?', uri) ? "&mode=" : "?mode=") + mode + "&lang=" + lang;
    
# url = "https://vsmobile.bet9ja.com/mobile/themes/?sk=bet9ja&t=b61c29e6-9348-4c58-af90-378760a74693&game=league_premier&pid=14001,14003,14011,14012,14014,14015,14016,14017&v=0&mode=Premier&lang=en_GB";
# url = "https://vsmobile.bet9ja.com/shopadmin/standings_view/league_data.php?pid=14003"

In [110]:
# new_df = pd.DataFrame({"home": ["for"], "away": ["che"]})
# new_df

In [111]:
# pd.get_dummies(new_df)

In [112]:
# sgd_clf.predict(pd.get_dummies(new_df))